In [1]:
spark

In [2]:
sparkDF = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("FileStore/tables/taxi_train.csv")

In [3]:
inputPath = "/FileStore/tables/taxi_train.csv"

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

In [5]:
mySchema = StructType() \
  .add("Records", ArrayType(StructType() \
                           .add("id", StringType()) \
                           .add("vendor_id", IntegerType()) \
                           .add("pickup_datetime", TimestampType()) \
                           .add("dropoff_datetime", TimestampType()) \
                           .add("passenger_count", IntegerType()) \
                           .add("pickup_longitude", FloatType()) \
                           .add("pickup_latitude", FloatType()) \
                           .add("dropoff_longitude", FloatType()) \
                           .add("dropoff_latitude", FloatType()) \
                           .add("store_and_fwd_flag", StringType()) \
                           .add("trip_duration", IntegerType())))
  


In [6]:
rawRecords = spark.readStream \
  .option("maxFilesPerTrigger", "100") \
  .schema(mySchema) \
  .csv(inputPath)

In [7]:
cloudTrailEvents = rawRecords \
  .select(explode("Records").alias("record"))

In [8]:
csvSchema = StructType([StructField("id", StringType(), True),
                        StructField("vendor_id", IntegerType(), True),
                        StructField("pickup_datetime", TimestampType(), True),
                        StructField("dropoff_datetime", TimestampType(), True),
                        StructField("passenger_count", IntegerType(), True),
                        StructField("pickup_longitude", FloatType(), True),
                        StructField("pickup_latitude", FloatType(), True),
                        StructField("dropoff_longitude", FloatType(), True),
                        StructField("dropoff_latitude", FloatType(), True),
                        StructField("store_and_fwd_flag", StringType(), True),
                        StructField("trip_duration", IntegerType(), True)])

  
staticInputDF = spark.read.csv('FileStore/tables/taxi_train.csv', header=True, schema=csvSchema)

display(staticInputDF)

In [9]:
staticInputDF.columns

In [10]:
staticInputDF.count()

In [11]:
staticInputDF.select('vendor_id').distinct().show()

In [12]:
display(staticInputDF.select('passenger_count').groupBy('passenger_count').count().orderBy("count", ascending=False))

In [13]:
staticInputDF.select(month('pickup_datetime')).distinct().orderBy('month(pickup_datetime)').show()

In [14]:
display(staticInputDF.select(dayofmonth('pickup_datetime')).groupBy('dayofmonth(pickup_datetime)').count().orderBy('dayofmonth(pickup_datetime)'))

In [15]:
staticInputDF.filter(year('pickup_datetime')== '2016').filter(dayofyear('pickup_datetime') >= 176).groupBy(dayofyear('pickup_datetime')).count().orderBy('dayofyear(pickup_datetime)').show()

In [16]:
display(staticInputDF.filter(year('pickup_datetime')== '2016').filter(dayofyear('pickup_datetime') >= 176).groupBy(dayofyear('pickup_datetime')).count().orderBy('dayofyear(pickup_datetime)'))

In [17]:
display(staticInputDF.select(dayofmonth('dropoff_datetime')).groupBy('dayofmonth(dropoff_datetime)').count().orderBy('dayofmonth(dropoff_datetime)'))

In [18]:
staticInputDF.select('vendor_id', 'passenger_count').groupBy('vendor_id').count().show()

In [19]:
staticInputDF.groupBy('store_and_fwd_flag').agg(avg('trip_duration')).show()

In [20]:
staticInputDF.groupBy('passenger_count').sum('trip_duration').collect()

In [21]:
from pyspark.sql import functions as F
staticInputDF.agg(F.max(staticInputDF.trip_duration)).show()

In [22]:
display(staticInputDF)

In [23]:
from pyspark.sql.functions import *

staticCountsDF = (
  staticInputDF
    .groupBy(
        staticInputDF.passenger_count)
    .count()
)

staticCountsDF.cache()

staticCountsDF.createOrReplaceTempView("static_counts")

In [24]:
%sql select passenger_count, sum(count) as total_count from static_counts group by passenger_count

In [25]:
staticCountsDF_1 = (
  staticInputDF
    .groupBy(
        staticInputDF.trip_duration)
    .count()
)

staticCountsDF_1.cache()

staticCountsDF_1.createOrReplaceTempView("static_counts_1")

In [26]:
%sql select trip_duration, sum(count) as total_count from static_counts_1 group by trip_duration

In [27]:
spark.sql("select count(*) from static_counts")

In [28]:
df = staticInputDF

In [29]:
df.select("id").where("trip_duration" > 100)